In [1]:
import numpy as np
import pandas as pd
import gpflow

from sklearn.model_selection import train_test_split

from gpflow.test_util import notebook_niter

In [2]:
import itertools
datafile = 'data/jass/rnd_01.csv'
col_names = [a+str(b) for (a,b) in itertools.product([farbe for farbe in 'HKSE'], [bild for bild in range(9)])]+["Geschoben", "Player", "Aktion"]
data = pd.read_csv(datafile, header=None, names=col_names)
data

,H0,H1,H2,H3,H4,H5,H6,H7,H8,K0,...,E2,E3,E4,E5,E6,E7,E8,Geschoben,Player,Aktion
0,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,1,1631,2
1,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,64310,6
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,16721,1
3,1,0,0,0,1,0,0,0,1,1,...,0,0,1,0,1,0,0,1,0,4
4,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,72620,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161689,0,1,0,0,0,1,0,0,0,0,...,0,1,0,1,1,0,0,1,55942,3
161690,0,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,6
161691,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,1,0,0,0,3
161692,0,1,0,1,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,60659,6


## Multiclass Classification (without _schieben_)

In [3]:
# select subset of data
target_classes = [0,1,2,3,4,5]
num_classes = len(target_classes)
zeroone = data[data['Aktion'].isin(target_classes)]
zeroone = zeroone.drop(columns='Player')
X = zeroone.loc[:,'H0':'Geschoben'].values.astype(float)
y = zeroone.loc[:,'Aktion']
D = X.shape[1]
# print distribution over classes
y.value_counts().sort_index().plot(kind='bar');
y = y.values.astype(float).reshape(-1,1)

## Function to evaluate and save a model

In [4]:
import pathlib
def evaluate(X_train, y_train, kernel, kernel_args, n_classes, inducing_every, name=None, path='./models/07_Jass_GPFlow_multi/', overwrite=False):
    
    if not name is None:
        _path = pathlib.Path(path)
        if not _path.is_dir():
            raise Exception('path to store model does not exist: %s' % path)
        _file = pathlib.Path(path+name)
        if _file.exists():
            if overwrite:
                _file.unlink()
            else:
                print('%s already trained... continue' % (str(path+name)))
                return
    # clear default tensorflow graph/session from previous trainings/predictions
    gpflow.reset_default_graph_and_session()
    
    kern = kernel(**kernel_args)
    
    print('Evaluating kernel: '+kern.name + ' (' + name+ ')')
    print('Datasets: Train: %i (%i inducing variables)' % (len(y_train), len(y_train)//inducing_every))
    
    
    #model
    invlink = gpflow.likelihoods.RobustMax(n_classes) # Robustmax inverse link function
    likelihood = gpflow.likelihoods.MultiClass(n_classes, invlink=invlink)  # Multiclass likelihood

    Z = X_train[::inducing_every].copy()
    m = gpflow.models.SVGP(
        X_train, y_train, kern=kern, likelihood=likelihood,
        Z=Z, num_latent=num_classes, whiten=True, q_diag=True)
    
    # train
    opt = gpflow.train.ScipyOptimizer(options={'maxls': 30,'ftol':1e-10, 'disp':True})
    opt.minimize(m, maxiter=notebook_niter(8000))
    
    # save model
    if not name is None:
        saver = gpflow.saver.Saver()
        saver.save(path+name, m)

In [7]:
def load_model(name, path='./models/07_Jass_GPFlow_multi/'):
    gpflow.reset_default_graph_and_session()
    print(path+name)
    return gpflow.saver.Saver().load(path+name)

def accuracy(m, X, y):
    return np.equal(y.reshape(-1), np.argmax(m.predict_y(X)[0], axis=1)).mean()

def create_name(name, train_size, inducing_every):
    return name+'_'+str(train_size)+'_'+str(train_size//inducing_every)

# Baseline using Squared Exponential on 500/250 training samples

In [9]:
# prepare data
_train_size = 500
_test_size = 10000
_inducing_every = 2
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=_train_size, test_size=_test_size, random_state=23)    

In [11]:
# traind and save
%time
_name = create_name('rbf', _train_size, _inducing_every)
r = evaluate(
     X_train=X_train,
     y_train=y_train,
     kernel= gpflow.kernels.RBF,
     kernel_args = {'input_dim': D},
     n_classes=num_classes,
     inducing_every=_inducing_every,
     name=_name, overwrite=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs
Evaluating kernel: SquaredExponential (rbf_500_250)
Datasets: Train: 500 (250 inducing variables)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 950.535920
  Number of iterations: 8000
  Number of functions evaluations: 8344


INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 950.535920
  Number of iterations: 8000
  Number of functions evaluations: 8344
/usr/local/lib/python3.6/dist-packages/gpflow/saver/serializers.py:37: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(pathname_or_file_like) as h5file:


In [18]:
# load and predict metrics
m = load_model(_name)
print('Log Marginal Likelihood: %.2f\nTest Accuracy: %.2f' % (m.compute_log_likelihood(), accuracy(m, X_test, y_test)))

Log Marginal Likelihood: -947.93
Test Accuracy: 0.69


# Testing Multiple Kernels 

In [27]:
kernels = {
    'rbf' : (gpflow.kernels.RBF, {'input_dim' : D}),
    'rbf_ard' : (gpflow.kernels.RBF, {'input_dim' : D, 'ARD':True}),
    'mat52' : (gpflow.kernels.Matern52, {'input_dim' : D}),
    'mat32' : (gpflow.kernels.Matern32, {'input_dim' : D}),
    'linear' : (gpflow.kernels.Linear, {'input_dim' : D}),
    'linear_ard' : (gpflow.kernels.Linear, {'input_dim' : D, 'ARD':True}),
    'arccosine' : (gpflow.kernels.ArcCosine, {'input_dim' : D}),
    'rq' : (gpflow.kernels.RationalQuadratic, {'input_dim' : D})
}

In [28]:
# prepare data
_train_size = 500
_test_size = 10000
_inducing_every = 2
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=_train_size, test_size=_test_size, random_state=23)    

In [29]:
# traind and save
for name, kernel in kernels.items():
    _name = create_name(name, _train_size, _inducing_every)
    r = evaluate(
         X_train=X_train,
         y_train=y_train,
         kernel= kernel[0],
         kernel_args= kernel[1],
         n_classes=num_classes,
         inducing_every=_inducing_every,
         name=_name)

./models/07_Jass_GPFlow_multi/rbf_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/rbf_ard_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/mat52_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/mat32_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/linear_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/linear_ard_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/arccosine_500_250 already trained... continue
./models/07_Jass_GPFlow_multi/rq_500_250 already trained... continue


In [30]:
for name, kernel in kernels.items():
    _name = name+'_'+str(_train_size)+'_'+str(_train_size//_inducing_every)
    m = load_model(_name)
    print('%s\nLog Marginal Likelihood: %.2f\nTest Accuracy: %.2f\n' % (_name, m.compute_log_likelihood(), accuracy(m, X_test,y_test)))

rbf_500_250
Log Marginal Likelihood: -947.93
Test Accuracy: 0.68

rbf_ard_500_250
Log Marginal Likelihood: -917.86
Test Accuracy: 0.69

mat52_500_250
Log Marginal Likelihood: -949.60
Test Accuracy: 0.68

mat32_500_250
Log Marginal Likelihood: -1007.74
Test Accuracy: 0.68

linear_500_250
Log Marginal Likelihood: -914.09
Test Accuracy: 0.66

linear_ard_500_250
Log Marginal Likelihood: -887.05
Test Accuracy: 0.65

arccosine_500_250
Log Marginal Likelihood: -2363.56
Test Accuracy: 0.54

rq_500_250
Log Marginal Likelihood: -954.65
Test Accuracy: 0.69



## Linear (ARD) and RBF (ARD) on 1000/333 training samples

In [45]:
# prepare data
_train_size = 1000
_test_size = 10000
_inducing_every = 3
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=_train_size, test_size=_test_size, random_state=23)  

In [35]:
kernels = {
    'rbf_ard' : (gpflow.kernels.RBF, {'input_dim' : D, 'ARD':True}),
    'linear_ard' : (gpflow.kernels.Linear, {'input_dim' : D, 'ARD':True})
}

In [ ]:
# traind and save
for name, kernel in kernels.items():
    _name = create_name(name, _train_size, _inducing_every)
    r = evaluate(X,y,
         kernel= kernel[0],
         kernel_args= kernel[1],
         n_classes=num_classes,
         X_train=X_train,
         y_train=y_train,
         inducing_every=_inducing_every,
         name=_name)

In [ ]:
for name, kernel in kernels.items():
    _name = name+'_'+str(_train_size)+'_'+str(_train_size//_inducing_every)
    m = load_model(_name)
    print('%s\nLog Marginal Likelihood: %.2f\nTest Accuracy: %.2f\n' % (_name, m.compute_log_likelihood(), accuracy(m, X_test,y_test)))

## RBF (ARD) on 2000/500 training samples

In [40]:
# prepare data
_train_size = 2000
_test_size = 10000
_inducing_every = 4
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=_train_size, test_size=_test_size, random_state=23)  

In [41]:
kernels = {
    'rbf_ard' : (gpflow.kernels.RBF, {'input_dim' : D, 'ARD':True})
}

In [42]:
# traind and save
for name, kernel in kernels.items():
    _name = create_name(name, _train_size, _inducing_every)
    r = evaluate(X,y,
         kernel= kernel[0],
         kernel_args= kernel[1],
         n_classes=num_classes,
         X_train=X_train,
         y_train=y_train,
         inducing_every=_inducing_every,
         name=_name)

./models/07_Jass_GPFlow_multi/rbf_ard_2000_500 already trained... continue


In [44]:
for name, kernel in kernels.items():
    _name = name+'_'+str(_train_size)+'_'+str(_train_size//_inducing_every)
    m = load_model(_name)
    print('%s\nLog Marginal Likelihood: %.2f\nTest Accuracy: %.2f\n' % (_name, m.compute_log_likelihood(), accuracy(m, X_test,y_test)))

rbf_ard_2000_500
Log Marginal Likelihood: -3006.32
Test Accuracy: 0.74

